In [17]:
# Import necessary libraries

import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, split_shuffle_data

In [18]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/colorectal_liver_metastases.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [19]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

# Process survival data
for model_name, values in data.items():  # Iterate through each model
    for dataset in ["train"]:  # Apply to train, val, and test data
        vital_days = [v["row"]["overall_survival_months"] for v in values[dataset]]  # Extract vital days
        vital_status = [v["row"]["vital_status"] for v in values[dataset]]  # Extract vital status
        survival = []
        
        for months, censor in zip(vital_days, vital_status):
            if months >= 24:
                survival.append(1)
            elif censor != 0 and months < 24:
                survival.append(0)
            else:
                survival.append(np.nan)

        items = data[model_name][dataset]
        for idx in range(len(items)):
            items[idx]["row"]["survival"] = survival[idx]  # Store survival status

In [20]:
print(np.unique(survival, return_counts=True))


(array([ 0.,  1., nan]), array([ 18, 176,   3]))


In [21]:
# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features
    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue
    
    print(f"Model: {model_name}") # Print the model name
    
    # Extract labels and features, filtering out nans
    all_labels = [v["row"]["survival"] for v in values["train"] if not np.isnan(v["row"]["survival"])]
    # Stack features for non-nan labels
    all_items = np.vstack([v["feature"] for v in values["train"] if not np.isnan(v["row"]["survival"])])

    print(len(all_items))
    # Average across multiple shuffle splits
    n_splits = 5
    split_scores = []
    
    for split in range(n_splits):
        # Get stratified indices for new train/val split with different random seeds
        train_items, train_labels, val_items, val_labels, test_items, test_labels = split_shuffle_data(
            all_items, all_labels, train_ratio=0.5, val_ratio=0.1, random_seed=10+split, stratify=True
        )

        # Train model with hyperparameter optimization
        best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels)
        
        # Get score for this split
        split_score = evaluate_model(best_model, test_items, test_labels)
        split_scores.append(split_score)
    
    # Average the scores across splits
    avg_score = np.mean(split_scores)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = avg_score

[I 2025-02-05 12:05:43,125] A new study created in memory with name: no-name-565422b7-f470-48ff-8faf-b0b058e8d2df
[I 2025-02-05 12:05:43,130] Trial 0 finished with value: 0.6388888888888888 and parameters: {'k': 29}. Best is trial 0 with value: 0.6388888888888888.
[I 2025-02-05 12:05:43,135] Trial 1 finished with value: 0.5833333333333333 and parameters: {'k': 12}. Best is trial 0 with value: 0.6388888888888888.
[I 2025-02-05 12:05:43,139] Trial 2 finished with value: 0.5972222222222222 and parameters: {'k': 11}. Best is trial 0 with value: 0.6388888888888888.
[I 2025-02-05 12:05:43,144] Trial 3 finished with value: 0.7083333333333333 and parameters: {'k': 42}. Best is trial 3 with value: 0.7083333333333333.
[I 2025-02-05 12:05:43,149] Trial 4 finished with value: 0.625 and parameters: {'k': 3}. Best is trial 3 with value: 0.7083333333333333.
[I 2025-02-05 12:05:43,154] Trial 5 finished with value: 0.6805555555555556 and parameters: {'k': 28}. Best is trial 3 with value: 0.708333333333

Model: FMCIBExtractor
194


[I 2025-02-05 12:05:43,323] Trial 31 finished with value: 0.5555555555555556 and parameters: {'k': 18}. Best is trial 10 with value: 0.7638888888888888.
[I 2025-02-05 12:05:43,331] Trial 32 finished with value: 0.7222222222222222 and parameters: {'k': 41}. Best is trial 10 with value: 0.7638888888888888.
[I 2025-02-05 12:05:43,340] Trial 33 finished with value: 0.6805555555555556 and parameters: {'k': 50}. Best is trial 10 with value: 0.7638888888888888.
[I 2025-02-05 12:05:43,349] Trial 34 finished with value: 0.6666666666666667 and parameters: {'k': 2}. Best is trial 10 with value: 0.7638888888888888.
[I 2025-02-05 12:05:43,358] Trial 35 finished with value: 0.5972222222222222 and parameters: {'k': 13}. Best is trial 10 with value: 0.7638888888888888.
[I 2025-02-05 12:05:43,367] Trial 36 finished with value: 0.7361111111111112 and parameters: {'k': 38}. Best is trial 10 with value: 0.7638888888888888.
[I 2025-02-05 12:05:43,376] Trial 37 finished with value: 0.6111111111111112 and pa

Model: CTFMExtractor
194


[I 2025-02-05 12:05:45,237] Trial 34 finished with value: 0.3888888888888889 and parameters: {'k': 2}. Best is trial 3 with value: 0.6388888888888888.
[I 2025-02-05 12:05:45,245] Trial 35 finished with value: 0.47222222222222227 and parameters: {'k': 13}. Best is trial 3 with value: 0.6388888888888888.
[I 2025-02-05 12:05:45,253] Trial 36 finished with value: 0.5277777777777778 and parameters: {'k': 38}. Best is trial 3 with value: 0.6388888888888888.
[I 2025-02-05 12:05:45,262] Trial 37 finished with value: 0.3194444444444445 and parameters: {'k': 25}. Best is trial 3 with value: 0.6388888888888888.
[I 2025-02-05 12:05:45,270] Trial 38 finished with value: 0.16666666666666666 and parameters: {'k': 7}. Best is trial 3 with value: 0.6388888888888888.
[I 2025-02-05 12:05:45,279] Trial 39 finished with value: 0.3194444444444445 and parameters: {'k': 24}. Best is trial 3 with value: 0.6388888888888888.
[I 2025-02-05 12:05:45,287] Trial 40 finished with value: 0.5277777777777778 and paramet

...

[I 2025-02-05 12:05:52,267] Trial 35 finished with value: 0.4444444444444444 and parameters: {'k': 13}. Best is trial 4 with value: 0.6527777777777778.
[I 2025-02-05 12:05:52,276] Trial 36 finished with value: 0.5972222222222222 and parameters: {'k': 38}. Best is trial 4 with value: 0.6527777777777778.
[I 2025-02-05 12:05:52,284] Trial 37 finished with value: 0.4305555555555556 and parameters: {'k': 25}. Best is trial 4 with value: 0.6527777777777778.
[I 2025-02-05 12:05:52,292] Trial 38 finished with value: 0.5972222222222223 and parameters: {'k': 7}. Best is trial 4 with value: 0.6527777777777778.
[I 2025-02-05 12:05:52,301] Trial 39 finished with value: 0.4305555555555556 and parameters: {'k': 24}. Best is trial 4 with value: 0.6527777777777778.
[I 2025-02-05 12:05:52,309] Trial 40 finished with value: 0.4861111111111111 and parameters: {'k': 37}. Best is trial 4 with value: 0.6527777777777778.
[I 2025-02-05 12:05:52,318] Trial 41 finished with value: 0.375 and parameters: {'k': 22}

Model: MerlinExtractor
194


[I 2025-02-05 12:05:53,946] Trial 35 finished with value: 0.4444444444444444 and parameters: {'k': 13}. Best is trial 34 with value: 0.6944444444444444.
[I 2025-02-05 12:05:53,954] Trial 36 finished with value: 0.5555555555555556 and parameters: {'k': 38}. Best is trial 34 with value: 0.6944444444444444.
[I 2025-02-05 12:05:53,962] Trial 37 finished with value: 0.4166666666666667 and parameters: {'k': 25}. Best is trial 34 with value: 0.6944444444444444.
[I 2025-02-05 12:05:53,971] Trial 38 finished with value: 0.5694444444444444 and parameters: {'k': 7}. Best is trial 34 with value: 0.6944444444444444.
[I 2025-02-05 12:05:53,979] Trial 39 finished with value: 0.4444444444444445 and parameters: {'k': 24}. Best is trial 34 with value: 0.6944444444444444.
[I 2025-02-05 12:05:53,988] Trial 40 finished with value: 0.5555555555555556 and parameters: {'k': 37}. Best is trial 34 with value: 0.6944444444444444.
[I 2025-02-05 12:05:53,997] Trial 41 finished with value: 0.4722222222222222 and pa

Model: ModelsGenExtractor
194


[I 2025-02-05 12:05:55,625] Trial 31 finished with value: 0.5 and parameters: {'k': 18}. Best is trial 18 with value: 0.6111111111111112.
[I 2025-02-05 12:05:55,633] Trial 32 finished with value: 0.4861111111111111 and parameters: {'k': 41}. Best is trial 18 with value: 0.6111111111111112.
[I 2025-02-05 12:05:55,642] Trial 33 finished with value: 0.5833333333333334 and parameters: {'k': 50}. Best is trial 18 with value: 0.6111111111111112.
[I 2025-02-05 12:05:55,651] Trial 34 finished with value: 0.4444444444444444 and parameters: {'k': 2}. Best is trial 18 with value: 0.6111111111111112.
[I 2025-02-05 12:05:55,659] Trial 35 finished with value: 0.5 and parameters: {'k': 13}. Best is trial 18 with value: 0.6111111111111112.
[I 2025-02-05 12:05:55,668] Trial 36 finished with value: 0.38888888888888895 and parameters: {'k': 38}. Best is trial 18 with value: 0.6111111111111112.
[I 2025-02-05 12:05:55,678] Trial 37 finished with value: 0.4444444444444444 and parameters: {'k': 25}. Best is 

In [22]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict)
fig.show() # Show the plot